<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso3/ciclo1/3_crispdm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&id=1li4ahmMhPo2cEUVqQKRDA9ahHp2py4Xb" width="100%">

# Cross Industry Standard Process for Data Mining
---

En este notebook veremos un ejemplo práctico de la metodología _Cross Industry Standard Process for Data Mining_ (CRISPDM).

<img src="https://drive.google.com/uc?export=view&id=1n2Sgt3If68rpeBSB8cIn1irJwc2J3Ilr" width="100%">

Este problema lo abordaremos con las siguientes librerías:

> **Nota**: este problema lo abordaremos con `tensorflow`, `numpy` y `pandas` para manejo de datos y modelamiento, y con `plotly` para visualización. Es importante tener en cuenta esto, ya que uno de los problemas que vamos a abordar a lo largo de este curso es saber cómo podemos integrar proyectos de machine learning independiente de las librerías o el lenguaje de programación usado.

In [ ]:
import plotly.express as px
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet import preprocess_input
from IPython.display import display

Veamos cada etapa de _CRISPDM_ con un ejemplo práctico:

## **1. Entendimiento del Negocio**
---

Una empresa de desarrollo de videojuegos móviles planea implementar un sistema de reconocimiento de imágenes para el análisis del juego **piedra, papel o tijeras** en tiempo real desde cualquier dispositivo móvil moderno. Para ello, recolectaron imágenes de las manos de distintas personas en las tres posturas del juego.

Al equipo de ciencia de datos se le encargó implementar un modelo que permita clasificar de forma automática las posturas a partir de imágenes.

<img src="https://drive.google.com/uc?export=view&id=1_59piBEtkGjktv1P6uDPCXr7NMJ0ZjRL" width="80%">

En la etapa de **entendimiento del negocio** normalmente debemos dar respuesta a las siguientes preguntas:

a. ¿Quién es el cliente? ¿En qué unidad de negocio se encuentra?

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Respuesta</b></font>
</summary>

En este caso los clientes son el equipo de desarrollo de videojuegos representados por un _product owner_ quien estableció qué tipo de solución de analítica se esperaba.
</details>

2. ¿Qué problemas de negocio se trata de solucionar?

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Respuesta</b></font>
</summary>

Se requiere un modelo que pueda ser embebido en dispositivos móviles y que permita clasificar de forma automática imágenes de posturas de manos.
</details>

3. ¿Qué soluciones de ciencia de datos queremos construir?

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Respuesta</b></font>
</summary>

De acuerdo al estado del arte en clasificación de imágenes y con el supuesto de que el modelo debe ser compacto para su integración en aplicaciones móviles, una de las mejores opciones son redes neuronales convolucionales.
</details>

4. ¿Cómo implementáremos el proyecto?

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Respuesta</b></font>
</summary>

Una de las mejores opciones es usar librerías como `tensorflow` para manipulación de datos, modelamiento y despliegue.
</details>

4. ¿Cómo se usará el resultado del proyecto?

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Respuesta</b></font>
</summary>

El modelo se exportará en formato `hdf5` para que posteriormente sea embebido en un dispositivo móvil con herramientas como `tflite`.
</details>

5. ¿Quiénes estarán trabajando en el proyecto?

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Respuesta</b></font>
</summary>

El equipo de desarrollo para este proyecto está dado por:

- Empresa de desarrollo de videojuegos:
  - Product owner con conocimientos del negocio.
  - Lider técnico con conocimientos técnicos de la aplicación.
  - Desarrollador móvil que se encargará de la integración.
- Equipo de ciencia de datos:
  - Lider de proyecto encargado de entender el problema y plantear soluciones de analítica.
  - Científico de datos encargado de implementar y entrenar los modelos.
</details>

5. ¿Qué métricas se usarán para medir el proyecto?

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Respuesta</b></font>
</summary>

La solución en este caso es muy técnica, por ello, las métricas están muy ligadas al desempeño del modelo. En especial se medirán 3 cosas:

- Tamaño del modelo (MB).
- Tiempo de inferencia (milisegundos).
- Accuracy del modelo.

Tenga en cuenta que las métricas no necesariamente tienen que ser directamente del modelo, por lo general se suelen utilizar indicadores claves de desempeño (KPI) que evalúan distintos aspectos y ganancias en un negocio (número de clientes, ingresos, entre otros).
</details>

6. ¿Cómo se ejecutará el proyecto?

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Respuesta</b></font>
</summary>

El proyecto se puede ejecutar siguiendo distintas estrategias de estructuración de proyectos, por ejemplo, con metodologías ágiles como scrum y con el apoyo de herramientas como [Azure DevOps](https://azure.microsoft.com/es-es/products/devops/) o [Jira](https://www.atlassian.com/es/software/jira).

La planeación es algo que queda a disposición del equipo de desarrollo, en muchos lugares incluso se mantiene una lista de tareas en un documento de excel compartido. En este caso manejaremos de forma simple la ejecución con una tabla de tiempos:

| Tarea | Tiempo |
| --- | --- |
| Entendimiento del negocio | 1 semana |
| Entendimiento de los datos | 1 semana |
| Preparación de los datos | 2 semanas |
| Modelado | 1 semana |
| Evaluación | 0.5 semanas |
| Despliegue | 1 semana |
</details>

## **2. Entendimiento de los Datos**
---

En la etapa de entendimiento de los datos procedemos con la carga de datos y el análisis exploratorio de los datos.

<img src="https://drive.google.com/uc?export=view&id=1JcDmzhnIbiO0187TrYBY44l__zedk9u7" width="80%">

Cargaremos los datos con `tensorflow_datasets`:

In [ ]:
ds = tfds.load("rock_paper_scissors")
display(ds)

Como podemos ver, se trata de un conjunto de imágenes de tamaño `(300, 300, 3)` donde ya tenemos la información particionada en entrenamiento y prueba. Vamos a extraer las particiones:

In [ ]:
train_ds = ds["train"]
test_ds = ds["test"]

Como parte del análisis exploratorio, podemos extraer una imagen del conjunto de datos y visualizarla para entender el tipo de imágenes que estamos manejando:

In [ ]:
sample = next(iter(train_ds.take(1)))
display(sample)

El resultado es un diccionario con la imagen y su etiqueta. Podemos visualizar la imagen:

In [ ]:
img = sample["image"].numpy()
label = sample["label"].numpy()
fig = px.imshow(img, title=f"Label = {label}")
fig.show()

Ahora, vamos a extraer todas las etiquetas del conjunto de entrenamiento para visualizar su distribución:

In [ ]:
train_labels = list(map(
        lambda x: float(x),
        train_ds.map(lambda sample: sample["label"])
        ))
display(train_labels[:10])

En total tenemos `2520` imágenes en el conjunto de entrenamiento:

In [ ]:
display(len(train_labels))

Veamos la distribución:

In [ ]:
values, counts = np.unique(train_labels, return_counts=True)
fig = px.bar(
    x=values,
    y=counts,
    title="Distribución de etiquetas",
    labels={
        "x": "Etiquetas",
        "y": "Conteo"
    }
)
fig.show()

Con esto, sabemos que el conjunto de datos está balanceado.

## **3. Preparación de los Datos**
---

La preparación de los datos busca implementar estrategias para estandarizar los datos y dejarlos listos para el entrenamiento de un modelo.

<img src="https://drive.google.com/uc?export=view&id=1TX1-Mc_-H46s_ypBjzCfPvPHDc1tNW4Y" width="80%">

Para el ejemplo que estamos desarrollando debemos aplicar dos estrategias de pre-procesamiento:

- Cambiar el tamaño de las imágenes para que coincidan con el modelo compacto que se usará en la aplicación móvil.
- Aplicar la estrategia de pre-procesamiento definida en el modelo que se implementará.

En este caso adaptaremos una red `MobileNet`, la cual requiere imágenes de tamaño `(224, 224, 3)`, para esto, usaremos la función `resize` de `tensorflow`, adicional a esto, aplicaremos la función `preprocess_input` definida para esta red neuronal.

In [ ]:
def preprocess(sample):
    img = tf.image.resize(sample["image"], (224, 224))
    prep_img = preprocess_input(img)
    return prep_img, sample["label"]

Aplicamos el preprocesamiento sobre el conjunto de datos:

In [ ]:
train_ds_prep = (
        train_ds
        .map(preprocess)
        .batch(64)
        )
display(train_ds_prep)

In [ ]:
test_ds_prep = (
        train_ds
        .map(preprocess)
        .batch(64)
        )
display(test_ds_prep)

## **4. Modelamiento**
---

En la etapa de modelamiento se implementa algún algoritmo predictivo y se hace el ajuste de parámetros e hiper-parámetros.

<img src="https://drive.google.com/uc?export=view&id=1i_fvvKhDA4OfVT2wl--NMJAiEx6xqH-M" width="80%">

En este caso definiremos la red neuronal como un modelo de `tf.keras`:

In [ ]:
class ImageClassifier(Model):
    def __init__(self):
        super(ImageClassifier, self).__init__()
        self.feature_extractor = MobileNet(
            include_top=False,
            input_shape=(224, 224, 3)
        )
        self.pool = GlobalAveragePooling2D()
        self.den1 = Dense(128, activation="relu")
        self.drop = Dropout(0.2)
        self.out = Dense(3, activation="softmax")


    def build(self, input_shape):
        self.feature_extractor.build(input_shape)
        self.pool.build((None, 7, 7, 1024))
        self.den1.build((None, 1024))
        self.drop.build((None, 128))
        self.out.build((None, 128))

        super(ImageClassifier, self).build(input_shape)

    def call(self, x):
        features = self.feature_extractor(x)
        pool = self.pool(features)
        h = self.drop(self.den1(pool))
        pred = self.out(h)
        return pred

In [ ]:
model = ImageClassifier()
model.build((None, 224, 224, 3))
model.summary()

Veamos la predicción sobre un batch de datos:

In [ ]:
y_pred = model.predict(train_ds_prep.take(1))
display(y_pred.shape)

El modelo predice la probabilidad de que una imagen sea piedra, papel o tijera; por ello las predicciones son una matriz con 3 filas.

Ahora, vamos a compilar el modelo para el problema correspondiente:

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss=SparseCategoricalCrossentropy(),
    metrics=["categorical_accuracy"]
)

Con esto, podemos entrenar el modelo:

In [ ]:
history = model.fit(train_ds_prep, epochs=10, validation_data=test_ds_prep)

Veamos cómo son las pérdidas de entrenamiento y prueba durante cada época del modelo:

In [ ]:
losses = pd.DataFrame(
        data={
            "loss": np.concatenate([
                history.history["loss"],
                history.history["val_loss"]
                ]),
            "epoch": np.concatenate([
                np.arange(10),
                np.arange(10)
                ]),
            "type": (["train"] * 10) + (["test"] * 10)
            }
        )
display(losses)

Generamos la visualización:

In [ ]:
fig = px.line(losses, x="epoch", y="loss", color="type")
fig.show()

Podemos ver que el modelo está correctamente ajustado con los hiper-parámetros que seleccionamos.

## **5. Evaluación**
---

En la etapa de evaluación evaluamos métricas de desempeño del modelo y del negocio.

<img src="https://drive.google.com/uc?export=view&id=13NxDATWLutsDvi3uNbvz0BVx9kXNNC_l" width="80%">

Veamos algunas métricas de desempeño del modelo, primero obtenemos las predicciones y las etiquetas de todo el conjunto de test:

In [ ]:
y_pred = []
y_test = []
for x_batch, y_batch in test_ds_prep:
    y_test.append(y_batch)
    y_pred.append(np.argmax(model.predict(x_batch), axis=1))

Veamos el accuracy:

In [ ]:
y_pred = np.concatenate(y_pred)
y_test = np.concatenate(y_test)

In [ ]:
y_pred

In [ ]:
y_test

In [ ]:
acc = Accuracy()(y_test, y_pred)
display(acc)

También es importante evaluar cuánto tiempo tarda el modelo en hacer inferencia, para ello, medimos el tiempo sobre un batch de datos:

In [ ]:
import time
t0 = time.time()
y_pred = model.predict(test_ds_prep.take(1))
t = ( time.time() - t0 ) * 1000
display(f"Tiempo en milisegundos: {t:.2f}")

## **6. Despliegue**
---

En la etapa de despliegue generalmente se genera un producto usable a partir del modelo o aplicación desarrollada.

<img src="https://drive.google.com/uc?export=view&id=1qcZeGf0p3mV3YOhuOfDZLexiKjMz77sZ" width="80%">

Existen varias formas de desplegar un modelo, de esto hablaremos más adelante en el curso. Por el momento, exportaremos el modelo en formato `hdf5` para un posterior despliegue:

In [ ]:
model.save_weights("model.weights.h5")

Con esto, tenemos un modelo preparado para ser embebido en aplicaciones móviles con un muy buen desempeño.

## Recursos Adicionales
---

Los siguientes enlaces corresponden a sitios donde encontrará información muy útil para profundizar en los temas vistos en este notebook:

- [Tensorflow](https://www.tensorflow.org/)
- [Plotly](https://plotly.com/)

## Créditos
---

**Profesor**

- [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/)

**Asistente docente**:

- [Juan S. Lara MSc](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/)

**Diseño de imágenes:**
- [Brian Chaparro Cetina](mailto:bchaparro@unal.edu.co).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*